In [ ]:
import cv2
import numpy as np
import math
import os

In [ ]:
class PerspectiveTransformer:
    """
    Class for performing perspective transformation on video frames.

    Args:
        source (str or int): Video source path or camera index.

    Attributes:
        source: Video source.
        frame_width: Width of video frames.
        frame_height: Height of video frames.
        capture: Video capture object.

    Methods:
        get_frame_size(): Get the frame width and height from the video source.
        init_gui(): Initialize GUI window and trackbars.
        on_trackbar_change(val): Callback function for trackbar changes.
        calculate_transformation_matrix(alpha, beta, gamma, focal_length, dist): Calculate the transformation matrix.
        apply_perspective_transformation(frame, transformation_matrix): Apply perspective transformation to the frame.
        process_video(): Process video frames and display the transformed result.
    """
    def __init__(self, image_path):
        """
        Initialize the PerspectiveTransformer.

        Parameters:
            source (str or int): Video source path or camera index.
        """
        self.image = cv2.imread(image_path)
        self.frame_width, self.frame_height = self.image.shape[1], self.image.shape[0]

        self.init_gui()

    def init_gui(self):
        """
        Initialize the GUI window and trackbars.
        """
        cv2.namedWindow("Result", cv2.WINDOW_NORMAL)
        cv2.createTrackbar("Alpha", "Result", 90, 180, self.on_trackbar_change)
        cv2.createTrackbar("Beta", "Result", 90, 180, self.on_trackbar_change)
        cv2.createTrackbar("Gamma", "Result", 90, 180, self.on_trackbar_change)
        cv2.createTrackbar("f", "Result", 500, 2000, self.on_trackbar_change)
        cv2.createTrackbar("Distance", "Result", 500, 2000, self.on_trackbar_change)

    def on_trackbar_change(self, val):
        """
        Callback function for trackbar changes.

        Args:
            val (int): Current trackbar value.
        """
        alpha = (cv2.getTrackbarPos("Alpha", "Result") - 90) * math.pi / 180
        beta = (cv2.getTrackbarPos("Beta", "Result") - 90) * math.pi / 180
        gamma = (cv2.getTrackbarPos("Gamma", "Result") - 90) * math.pi / 180
        focal_length = cv2.getTrackbarPos("f", "Result")
        dist = cv2.getTrackbarPos("Distance", "Result")

        try:
            transformation_matrix = self.calculate_transformation_matrix(alpha, beta, gamma, focal_length, dist)

            processed_frame = self.apply_perspective_transformation(self.image, transformation_matrix)
            cv2.imshow("Result", processed_frame)
        except Exception as e:
            print(f"Error: {e}")

    def calculate_transformation_matrix(self, alpha, beta, gamma, focal_length, dist):
        """
        Calculate the perspective transformation matrix based on the provided parameters.

        Args:
            alpha (float): Alpha angle in radians.
            beta (float): Beta angle in radians.
            gamma (float): Gamma angle in radians.
            focal_length (int): Focal length.
            dist (int): Distance.

        Returns:
            numpy.ndarray: Transformation matrix.
        """
        
        image_size = (self.frame_width, self.frame_height)
        w, h = image_size

        a1 = np.array([[1, 0, -w / 2],
                    [0, 1, -h / 2],
                    [0, 0, 0],
                    [0, 0, 1]], dtype=np.float32)

        rx = np.array([[1, 0, 0, 0],
                    [0, math.cos(alpha), -math.sin(alpha), 0],
                    [0, math.sin(alpha), math.cos(alpha), 0],
                    [0, 0, 0, 1]], dtype=np.float32)

        ry = np.array([[math.cos(beta), 0, -math.sin(beta), 0],
                    [0, 1, 0, 0],
                    [math.sin(beta), 0, math.cos(beta), 0],
                    [0, 0, 0, 1]], dtype=np.float32)

        rz = np.array([[math.cos(gamma), -math.sin(gamma), 0, 0],
                    [math.sin(gamma), math.cos(gamma), 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]], dtype=np.float32)

        r = np.dot(np.dot(rx, ry), rz)

        t = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, dist],
                    [0, 0, 0, 1]], dtype=np.float32)

        k = np.array([[focal_length, 0, w / 2, 0],
                    [0, focal_length, h / 2, 0],
                    [0, 0, 1, 0]], dtype=np.float32)

        transformation_matrix = np.dot(np.dot(np.dot(k, t), r), a1)
        return transformation_matrix


    def apply_perspective_transformation(self, frame, transformation_matrix):
        """
        Apply the calculated perspective transformation to the frame.

        Args:
            frame (numpy.ndarray): Input frame.
            transformation_matrix (numpy.ndarray): Transformation matrix.

        Returns:
            numpy.ndarray: Processed frame after perspective transformation.
        """
        image_size = (self.frame_width, self.frame_height)
        processed_frame = cv2.warpPerspective(frame, transformation_matrix, image_size,
                                            flags=cv2.INTER_CUBIC + cv2.WARP_INVERSE_MAP)
        return processed_frame
    
    def process_image(self):
        while True:
            # self.update_perspective(0)
            self.on_trackbar_change(0)

            key = cv2.waitKey(1)
            if key == 27:  # Press 'Esc' to exit
                break

        cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    print("Perspective Transformation Application")
    print("Use the following controls to adjust parameters:")
    print(" - Alpha, Beta, Gamma: Adjust perspective angles")
    print(" - f: Adjust focal length")
    print(" - Distance: Adjust distance")
    print("Press 'Esc' to exit.")

    image_path = 'image.jpg'
    source = 0 if not os.path.isfile(image_path) else image_path

    try:
        perspective_transformer = PerspectiveTransformer(source)
        perspective_transformer.process_image()
    except Exception as e:
        print(f"An error occurred: {e}")